In [1]:
import os
import io
import gzip
import glob
import numpy as np
import pandas as pd
import tarfile
import matplotlib.pyplot as plt

from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import fastavro

from astropy.time import Time
from astropy.io import fits
import aplpy
%matplotlib inline

In [198]:
## Download ZTF data to epyc directly

#! curl -o /epyc/users/mmckay/ztf_public_20200416.tar.gz https://ztf.uw.edu/alerts/public/ztf_public_20200416.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6086M  100 6086M    0     0   276M      0  0:00:21  0:00:20  0:00:01  213M013M    0     0   275M      0  0:00:22  0:00:21  0:00:01  198M0  0:00:21  0:00:21 --:--:--  204M


In [6]:
#Copy ROSAT catalog from epyc to local directory
#os.system('cp  /epyc/data/rosat_2rxs/cat2rxs.fits /epyc/users/mmckay/')

0

# ROSAT dataframe

In [3]:
# Load ROSAT catalog
rosat_fits = fits.open('/epyc/users/mmckay/cat2rxs.fits')

# Make ROSAT data into a pandas dataframe
rosat_data = rosat_fits[1].data
dfx = pd.DataFrame(rosat_data)

# exclude sources that are not observable
dfx = dfx[dfx.DEC_DEG >= -30] 
#dfx

# append ROSAT error position [arcsec] 
dfx['err_pos_arcsec'] = np.sqrt(((dfx.XERR*45)**2.+ (dfx.YERR*45)**2.) + 0.6**2.)
dfx.err_pos_arcsec

0         12.209580
1         13.745840
2         11.233130
3         28.711039
4          5.574358
            ...    
104222    18.148684
104223    19.329465
104224    24.463706
104225    14.669044
104226    33.828258
Name: err_pos_arcsec, Length: 103351, dtype: float64

In [4]:
# List of ROSAT RA and DEC
rosat_ra_list = dfx.RA_DEG
rosat_dec_list = dfx.DEC_DEG

# Extract avro data from ZTF

In [199]:
## Extract avro data from the tar file to 
## *current working directory(Where this notebook is located)*
#tar_archive = '/epyc/users/mmckay/ztf_public_20200416.tar.gz'
#output_dir = tar_archive.split('/')[-1].split('.')[-3]
#archive = tarfile.open(tar_archive,'r:gz')
#archive.extractall(path=output_dir)
#archive.close()

In [4]:
#Count packets
def find_files(root_dir):
    for dir_name, subdir_list, file_list in os.walk(root_dir, followlinks=True):
        for fname in file_list:
            if fname.endswith('.avro'):
                yield dir_name+'/'+fname

In [5]:
ztf_data_dir = '/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200416/'
print('{} has {} avro files'.format(ztf_data_dir, len(list(find_files(ztf_data_dir)))))

/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200416/ has 140511 avro files


In [10]:
# Make list of avro files
fname_list = glob.glob('{}/*avro'.format(ztf_data_dir))

In [74]:
#Make packet into a dataframe
#def make_dataframe(packet):
#    df = pd.DataFrame(packet['candidate'], index=[0])
#    df_prv = pd.DataFrame(packet['prv_candidates'])
#    return pd.concat([df,df_prv], ignore_index=True)


def extract_avro_data(fname):
    with open(fname,'rb') as f:
        freader = fastavro.reader(f)
        schema = freader.writer_schema # freader.schema - depercated 
    
        for packet in freader:
            avro_ra = packet['candidate']['ra']
            avro_dec = packet['candidate']['dec']
            #print(packet.keys())
    # Make packet into a dataframe   
    #dflc = make_dataframe(packet)
    #avro_ra = dflc.ra[0]
    #avro_dec = dflc.dec[0]
    return avro_ra, avro_dec
        

# Multiprocessing - In-progress

In [269]:
# Use multiprocessing to speed up avro data extraction
#from multiprocessing import Pool
#from tqdm import tqdm
#from tqdm.notebook import tqdm
#
#
#def extract_avro_radec(fname_list):
#    avro_ra_list = []
#    avro_dec_list = []
#    for avro in tqdm(fname_list):
#        print(avro)
#        ra, dec = extract_avro_data(avro)
#        avro_ra_list.append(ra)
#        avro_dec_list.append(dec)
#
#def extract_avro_mp(fname_list):
#    chunks = [fname_list[i::5] for i in range(5)] #split the list ogf avro files into 5 seperate chucks
#    print(chunks)
#    
#    pool = Pool(processes=5)
#    
#    result = pool.map(extract_avro_radec, chunks)
#    print(result)
#    return result


In [8]:
#r = extract_avro_mp(fname_list[:4])

In [275]:
#test = fname_list[:5]
#chunks = [test[i::5] for i in range(5)]
#print(chunks[0][0])
#a = extract_avro_radec(chunks[0][0])
#a

/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200402/1187128334515010008.avro


In [276]:
#chunks[1]

['/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200402/1187128331015010031.avro']

# Skycoord

In [14]:
from astropy.coordinates import SkyCoord  # High-level coordinates
#from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
#from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

In [15]:
# Put ROSAT ra and dec list in SkyCoord [degrees]
rosat_skycoord = SkyCoord(ra = rosat_ra_list, dec=rosat_dec_list, frame='icrs', unit=(u.deg))
rosat_skycoord

<SkyCoord (ICRS): (ra, dec) in deg
    [(183.50286557,  87.19713271), (169.10778487,  87.19988261),
     (168.08524479,  87.43678965), ..., (356.84304802, -29.89944039),
     (356.18179242, -29.94573029), (358.9696836 , -29.97436576)]>

In [233]:
# Check coordinates object is an array
rosat_skycoord.isscalar

False

In [19]:
# Put avro data in SkyCoord 
#ra = dflc.ra[0]
#dec = dflc.dec[0]
#ra = ztf_ra_list
#dec = ztf_dec_list
avro_skycoord = SkyCoord(ra=avro_ra_list, dec=avro_dec_list, frame='icrs', unit=(u.deg))
#c = SkyCoord(ra=ra, dec=dec, frame='icrs', unit=(u.hourangle, u.deg))
avro_skycoord

<SkyCoord (ICRS): (ra, dec) in deg
    [(104.6744271, -15.3390104), (106.247876 , -20.5182166),
     (106.1343784, -19.6293488), (106.4677208, -19.4312591),
     (111.2721758, -16.7549467), (104.902724 , -16.4319291),
     (105.9973217, -18.7481326), (107.4867928, -17.9728991),
     (105.2964855, -15.9753838), (105.1938004, -17.724625 )]>

In [20]:
# Check coordinates object is an array
avro_skycoord.isscalar

False

In [88]:
# Crossmatch function

def ztf_rosat_crossmatch(ztf_ra, ztf_dec, rosat_skycoord, rosat_pos_err):
    """
    Cross match ZTF and ROSAT data in astropy.SkyCoords
    
    Parameters:
    
        ztf_ra: float or list of float
        
        ztf_dec: float or list of float
        
        rosat_ra_list: pandas.series
        
        rosat_dec_list: pandas/series
        
    Return:
        
        ztf_rosat_ra_match: float
        
        ztf_rosat_dec_match: float
        
        match: tuple
            Output from Skycoord.match_to_catalog_sky
        
        
    """
    # Input avro data ra and dec in SkyCoords
    avro_skycoord = SkyCoord(ra=ztf_ra, dec=ztf_dec, frame='icrs', unit=(u.deg))
    
    
    
    
    # Finds the nearest ROSAT source's coordinates to the avro files ra[deg] and dec[deg]
    match = avro_skycoord.match_to_catalog_sky(catalogcoord=rosat_skycoord, nthneighbor=1)
    #i = 0
    match_idx = match[0]
    match_sep2d = match[2]
    
    if match_sep2d <= rosat_pos_err[match_idx] * 0.000277778:
        match_result = 'Good! sep2d={} deg'.format(match_sep2d)
    else:
        match_result = 'Not so good sep2d={} deg'.format(match_sep2d)
    
    print('ZTF   RA: {}, DEC: {}'.format(avro_skycoord.ra, avro_skycoord.dec))
    print('ROSAT RA: {}, DEC: {}'.format(rosat_skycoord.ra[match_idx], rosat_skycoord.dec[match_idx]))
    print('Match Result: {}\n'.format(match_result))
    #i += 1
    
    
    #for idx, sep2d in tqdm(zip(match[0], match[2])):
    #    if sep2d <= rosat_pos_err[idx] * 0.000277778:
    #        match_result = 'Good! sep2d={} deg'.format(sep2d)
    #    else:
    #        match_result = 'Not so good sep2d={} deg'.format(sep2d)
    #        
    #    
    #    print('ZTF   RA: {}, DEC: {}'.format(avro_skycoord.ra[i], avro_skycoord.dec[i]))
    #    print('ROSAT RA: {}, DEC: {}'.format(rosat_skycoord.ra[idx], rosat_skycoord.dec[idx]))
    #    print('Match Result: {}\n'.format(match_result))
    #    i += 1
        
    
    
    # Index match indice with rosat coordinates
    #rosat_ra_match = rosat_skycoord.ra.value[idx]
    #rosat_dec_match = rosat_skycoord.dec.value[idx]
    
    #print('ZTF   RA: {} DEC: {} [deg]'.format(ra,dec))
    #print('ROSAT RA: {} DEC: {} [deg]'.format(rosat_ra_match, rosat_dec_match))
    #print('On-Sky seperation between the closet match {} \n'.format(match[1]))
    #return rosat_ra_match, rosat_dec_match, match
    
    

In [86]:
%%time
# Input rosat data ra list and dec list in SkyCoords
rosat_skycoord = SkyCoord(ra = rosat_ra_list, dec=rosat_dec_list, frame='icrs', unit=(u.deg))



CPU times: user 7.26 s, sys: 267 ms, total: 7.52 s
Wall time: 7.61 s


In [7]:
#%%time
#for avro in tqdm(fname_list[:]):
#    ra, dec = extract_avro_data(avro)
#    print('RA: {}, DEC: {}'.format(ra, dec))
#    
#    ztf_rosat_crossmatch(ra, dec, rosat_skycoord, dfx.err_pos_arcsec)

In [61]:
%%time
ztf_rosat_crossmatch(avro_ra_list, avro_dec_list, rosat_skycoord, dfx.err_pos_arcsec)

ZTF   RA: 104.6744271 deg, DEC: -15.3390104 deg
ROSAT RA: 104.64307171107367 deg, DEC: -15.300799410405327 deg
Match Result: Good! sep2d=0.0008504983396979236 deg

ZTF   RA: 106.247876 deg, DEC: -20.5182166 deg
ROSAT RA: 106.30238473523903 deg, DEC: -20.34752208139387 deg
Match Result: Not so good sep2d=0.003109708020907392 deg

ZTF   RA: 106.1343784 deg, DEC: -19.6293488 deg
ROSAT RA: 106.41869221135951 deg, DEC: -19.65370774930093 deg
Match Result: Good! sep2d=0.004692773991304001 deg

ZTF   RA: 106.4677208 deg, DEC: -19.4312591 deg
ROSAT RA: 106.41869221135951 deg, DEC: -19.65370774930093 deg
Match Result: Good! sep2d=0.003965323588956156 deg

ZTF   RA: 111.2721758 deg, DEC: -16.7549467 deg
ROSAT RA: 111.15870596145815 deg, DEC: -16.741814045701215 deg
Match Result: Good! sep2d=0.0019102127634821862 deg

ZTF   RA: 104.902724 deg, DEC: -16.4319291 deg
ROSAT RA: 105.29124629139554 deg, DEC: -16.49637427909155 deg
Match Result: Not so good sep2d=0.006599497171718853 deg

ZTF   RA: 105.

In [28]:
# testing
match = avro_skycoord.match_to_catalog_sky(catalogcoord=rosat_skycoord, nthneighbor=1)
idx = match[0]

In [29]:
idx

array([90083, 95500, 90150, 90150, 90184, 90101, 90134, 90121, 90097,
       90122])

# Test script

In [ ]:
import pandas as pd
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter
import fastavro
import aplpy
from astropy.coordinates import SkyCoord
import astropy.units as u
import glob

# Extract .avro RA and Dec 
def extract_avro_data(fname):
    """
    Parameters:
                fname: str
                
    
    """
    with open(fname,'rb') as f:
        freader = fastavro.reader(f)
        schema = freader.writer_schema # freader.schema - depercated 
    
        for packet in freader:
            avro_ra = packet['candidate']['ra']
            avro_dec = packet['candidate']['dec']

    return avro_ra, avro_dec



def ztf_rosat_crossmatch(avro_ra, avro_dec, rosat_skycoord, rosat_pos_err):
    """
    
    Cross match ZTF and ROSAT data using astropy.coordinates.SkyCoord
    
    Parameters:
                avro_ra: float or list of float
                
                avro_dec: float or list of float
                
                rosat_skycoord: float or list 
                
                rosat_ra_list: pandas.series
                
                rosat_dec_list: pandas/series
                
    Return:
        
                ztf_rosat_ra_match: float
                
                ztf_rosat_dec_match: float
                
                match: tuple
                    Output from Skycoord.match_to_catalog_sky
    
    """
    

    # Input avro data ra and dec in SkyCoords
    avro_skycoord = SkyCoord(ra=avro_ra, dec=avro_dec, frame='icrs', unit=(u.deg))
    
    
    # Finds the nearest ROSAT source's coordinates to the avro files ra[deg] and dec[deg]
    match = avro_skycoord.match_to_catalog_sky(catalogcoord=rosat_skycoord, nthneighbor=1, )
    match_idx = match[0]
    match_sep2d = match[2]
    
    if match_sep2d <= rosat_pos_err[match_idx] * 0.000277778:
        match_result = 'Good! sep2d={} deg'.format(match_sep2d)
        print('ZTF   RA: {}, DEC: {}'.format(avro_skycoord.ra, avro_skycoord.dec))
        print('ROSAT RA: {}, DEC: {}'.format(rosat_skycoord.ra[match_idx], rosat_skycoord.dec[match_idx]))
        print('Match Result: {}\n'.format(match_result)) 

    else:
        pass
        #match_result = 'Not so good sep2d={} deg'.format(match_sep2d)
    
    
    return avro_skycoord.ra, avro_skycoord.dec, rosat_skycoord.ra[match_idx], rosat_skycoord.dec[match_idx], match_sep2d
    
    
# ROSAT data********
    
    
# Open ROSAT catalog
rosat_fits = fits.open('/epyc/users/mmckay/cat2rxs.fits')

# Make ROSAT data into a pandas dataframe
rosat_data = rosat_fits[1].data
dfx = pd.DataFrame(rosat_data)

# exclude sources that are not observable
dfx = dfx[dfx.DEC_DEG >= -30] 

# List of ROSAT RA and DEC
rosat_ra_list = dfx.RA_DEG
rosat_dec_list = dfx.DEC_DEG

# List ROSAT error position [arcsec] 
dfx['err_pos_arcsec'] = np.sqrt(((dfx.XERR*45)**2.+ (dfx.YERR*45)**2.) + 0.6**2.)

# Put ROSAT ra and dec list in SkyCoord [degrees]
rosat_skycoord = SkyCoord(ra=rosat_ra_list, dec=rosat_dec_list, frame='icrs', unit=(u.deg))


# .avro data***********


# Move .avro files into a List
ztf_data_dir = '/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200402'
avro_fname_list = glob.glob('{}/*.avro'.format(ztf_data_dir))

# Make a list of the data
avro_ra_list = []
avro_dec_list = []

rosat_ra_list = []
rosat_dec_list = []

sep2d_list = []

# Run crossmatch on .avro files
for avro in tqdm(fname_list):
    
    # Extract ra and dec values from avro files
    avro_ra, avro_dec = extract_avro_data(avro)
    
    # Run crossmath functions
    avro_ra_match, avro_dec_match, rosat_ra_match, rosat_dec_match, match_2dsep = ztf_rosat_crossmatch(avro_ra, avro_dec, rosat_skycoord, dfx.err_pos_arcsec)
    
    
    avro_ra_list.append(avro_ra_match)
    avro_dec_list.append(avro_dec_match)
    rosat_ra_list.append(rosat_ra_match)
    rosat_dec_list.append(rosat_dec_match)
    sep2d_list.append(match_2dsep)
    
    
    

    
    



In [111]:
# Make datafame to store .avro and rosat match RA, Dec and 2d seperation angle
column_names = ["avro_RA_[deg]",  "avro_Dec_[deg]", "ROSAT_RA_[deg]", "ROSAT_Dec_[deg]", "2D_sep_[deg]"]
crossmatch_df = pd.DataFrame(columns=column_names)

crossmatch_df['avro_RA_[deg]'] = avro_ra_list
crossmatch_df['avro_Dec_[deg]'] = avro_dec_list
crossmatch_df['ROSAT_RA_[deg]'] = rosat_ra_list
crossmatch_df['ROSAT_Dec_[deg]'] = rosat_dec_list
crossmatch_df['2D_sep_[deg]'] = sep2d_list
    

In [6]:
#%%time
#!python3 ztf_rosat_crossmatch.py -ztf_path='/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200402/' > output.txt



In [200]:
%%time
!python3 ztf_rosat_crossmatch.py -ztf_path='/epyc/users/mmckay/alert_stream_crossmatch/ztf_public_20200416/' > output1.txt



/epyc/opt/anaconda-2019/lib/python3.7/site-packages/ipykernel/kernelbase.py:19: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  from jupyter_client.session import utcnow as now
CPU times: user 1min 44s, sys: 29.2 s, total: 2min 14s
Wall time: 57min 54s


In [141]:
SkyCoord

astropy.coordinates.sky_coordinate.SkyCoord